# 实例描述
通过定义两个矩阵变量相乘来演示使用gradients求梯度。

在反向传播过程中，神经网络需要对每一个loss对应的学习参数**求偏导**，算出的这个值也叫梯度。它是通过tf.gradients函数来实现的。tf.gradients函数里第一个参数为求导公式的结果，第二个参数为指定公式中的哪个变量来求偏导。

In [6]:
import tensorflow as tf

w1 = tf.Variable([[1.,2.]])
w2 = tf.Variable([[3.,4]])

y = tf.matmul(w1,[[9.],[10]])
grads = tf.gradients(y,[w1])  # 求w1的梯度

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    gradval = sess.run(grads)
    print(gradval)

[array([[ 9., 10.]], dtype=float32)]


上面例子中，由于y是由w1与[[9],[10]]相乘而来，所以其倒数也就是[[9],[10]]（即斜率）。

# 使用gradients对多个式子求多变量偏导
有两个OP，4个参数，使用gradients同时为两个式子4各参数求梯度。

In [2]:
import tensorflow as tf

tf.reset_default_graph()

w1 = tf.get_variable('w1',shape=[2])
w2 = tf.get_variable('w2',shape=[2])

w3 = tf.get_variable('w3',shape=[2])
w4 = tf.get_variable('w4',shape=[2])

y1 = w1 + w2 + w3
y2 = w3 + w4

# grad_ys求梯度的输入值
gradients = tf.gradients([y1,y2],[w1,w2,w3,w4],grad_ys=[tf.convert_to_tensor([1.,2]),tf.convert_to_tensor([3.,4])])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(gradients))


[array([1., 2.], dtype=float32), array([1., 2.], dtype=float32), array([4., 6.], dtype=float32), array([3., 4.], dtype=float32)]


这里使用了tf.gradients函数的第三个参数，即给定公式结果的值，来求参数偏导，这里相当于y1为[1.，2.]、y2为[3.，4.]。对于y1来讲，求关于w1的偏导时，会认为w2和w3为常数，所以w1和w2的导数为0，即w1的梯度就为[1.，2.]。同理可以得出w2和w3均为[1.，2.]，接着求y2的偏导数，得到w3与w4均为[3.，4.]。然后将两个式子中的w3结果累加起来，所以w3就为[4.，6.]。

# 演示梯度停止
演示梯度停止的用法，并观察当变量设置梯度停止后，对其求梯度的结果。

对于反向传播过程中某种特殊情况需要停止梯度的运算时，在TensorFlow中提供了一个tf.stop_gradient函数，被它定义过的节点将没有梯度运算功能。

In [3]:
a = w1 + w2 
a_stoped = tf.stop_gradient(a) #令a梯度停止
y3 = a_stoped + w3

gradients = tf.gradients([y1,y2],[w1,w2,w3,w4],grad_ys=[tf.convert_to_tensor([1.,2]),tf.convert_to_tensor([3.,4])])
gradients2 = tf.gradients(y3,[w1,w2,w3],grad_ys=tf.convert_to_tensor([1.,2]))
print(gradients2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(gradients))
    print(sess.run(gradients2))


[None, None, <tf.Tensor 'gradients_2/grad_ys_0:0' shape=(2,) dtype=float32>]
[array([1., 2.], dtype=float32), array([1., 2.], dtype=float32), array([4., 6.], dtype=float32), array([3., 4.], dtype=float32)]


TypeError: Fetch argument None has invalid type <class 'NoneType'>

可以看到程序报错，并且在出错前显示了gradients2的内容，w1和w2对应的位置都为None,这是由于梯度被停止了。后面的程序试图去求一个None的梯度，所以报错了。

在定义一个gradients3，只求存在的梯度。

In [4]:
gradients3 = tf.gradients(y3,[w3],grad_ys=tf.convert_to_tensor([1.,2]))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(gradients))
    print(sess.run(gradients3))

[array([1., 2.], dtype=float32), array([1., 2.], dtype=float32), array([4., 6.], dtype=float32), array([3., 4.], dtype=float32)]
[array([1., 2.], dtype=float32)]
